# 连接到 MCP 服务器

In [3]:
from hello_agents.tools import MCPTool, A2ATool, ANPTool

# 1. MCP：访问工具
mcp_tool = MCPTool()
result = mcp_tool.run({
    "action": "call_tool",
    "tool_name": "add",
    "arguments": {"a": 10, "b": 20}
})
print(f"MCP计算结果: {result}")  # 输出: 30.0

INFO:mcp.server.lowlevel.server:Processing request of type ListToolsRequest
INFO:mcp.server.lowlevel.server:Processing request of type CallToolRequest
INFO:mcp.server.lowlevel.server:Processing request of type ListToolsRequest


🧠 使用内存传输: HelloAgents-BuiltinServer
🔗 连接到 MCP 服务器...
✅ 连接成功！
🔌 连接已断开
🧠 使用内存传输: HelloAgents-BuiltinServer
🔗 连接到 MCP 服务器...
✅ 连接成功！
🔌 连接已断开
MCP计算结果: 工具 'add' 执行结果:
30.0


In [4]:
# 2. ANP：服务发现
anp_tool = ANPTool()
anp_tool.run({
    "action": "register_service",
    "service_id": "calculator",
    "service_type": "math",
    "endpoint": "http://localhost:8080"
})
services = anp_tool.run({"action": "discover_services"})
print(f"发现的服务: {services}")


发现的服务: 找到 1 个服务:

服务ID: calculator
  名称: calculator
  类型: math
  端点: http://localhost:8080




In [ ]:
# 3. A2A：智能体通信
a2a_tool = A2ATool("http://localhost:5000")
print("A2A工具创建成功")

A2A工具创建成功


## 调用工具

In [19]:
import asyncio
import os
from hello_agents.protocols import MCPClient

async def use_tools():
    client = MCPClient(["npx", "-y", "@modelcontextprotocol/server-filesystem", "."])

    async with client:
        # 读取文件
        result = await client.call_tool("read_file", {"path": "my_README.md"})
        print(f"文件内容：\n{result}\n")

        # 列出目录
        result = await client.call_tool("list_directory", {"path": "."})
        print(f"当前目录文件：\n{result}")

        # 写入文件
        result = await client.call_tool("write_file", {
            "path": "output.txt",
            "content": "Hello from MCP!"
        })
        print(f"写入结果：{result}")


async def safe_tool_call():
    client = MCPClient(["npx", "-y", "@modelcontextprotocol/server-filesystem", "."])

    async with client:
        try:
            # 尝试读取可能不存在的文件
            result = await client.call_tool("read_file", {"path": "nonexistent.txt"})
            print(result)
        except Exception as e:
            print(f"工具调用失败: {e}")
            # 可以选择重试、使用默认值或向用户报告错误

In [18]:
await use_tools()

📝 使用 Stdio 传输 (命令): npx -y @modelcontextprotocol/server-filesystem .
🔗 连接到 MCP 服务器...
✅ 连接成功！
文件内容：
这是Helloagents第十章代码仓库的演示

当前目录文件：
[FILE] 02_Connect2MCP.py
[FILE] my_README.md
[FILE] my_mcp_server.py
[FILE] output.txt
[FILE] test.ipynb
写入结果：Successfully wrote to output.txt
🔌 连接已断开


In [21]:
await safe_tool_call()

📝 使用 Stdio 传输 (命令): npx -y @modelcontextprotocol/server-filesystem .
🔗 连接到 MCP 服务器...
✅ 连接成功！
工具调用失败: ENOENT: no such file or directory, open '/mnt/d/studying-code/Large-Model-Application-Development/hello-agents/chapter10/nonexistent.txt'
🔌 连接已断开


## 访问资源

In [46]:
client = MCPClient(["npx", "-y", "@modelcontextprotocol/server-filesystem", "."])
# 列出可用资源
resources = client.list_resources()
print(f"可用资源：{[r['uri'] for r in resources]}")

# 读取资源
resource_content = client.read_resource("file:///path/to/resource")
print(f"资源内容：{resource_content}")

📝 使用 Stdio 传输 (命令): npx -y @modelcontextprotocol/server-filesystem .


TypeError: 'coroutine' object is not iterable

In [47]:
async def safe_tool_call():
    client = MCPClient(["npx", "-y", "@modelcontextprotocol/server-filesystem", "."])

    async with client:
        try:
            resources = client.list_resources()
            print(f"可用资源：{[r['uri'] for r in resources]}")
            print(resources)
        except Exception as e:
            print(f"工具调用失败: {e}")
            # 可以选择重试、使用默认值或向用户报告错误

In [48]:
await safe_tool_call()

📝 使用 Stdio 传输 (命令): npx -y @modelcontextprotocol/server-filesystem .
🔗 连接到 MCP 服务器...
✅ 连接成功！
工具调用失败: 'coroutine' object is not iterable
🔌 连接已断开


/tmp/ipykernel_3661357/1399694477.py:1: RuntimeWarning: coroutine 'MCPClient.list_resources' was never awaited
  await safe_tool_call()


## MCP 传输方式详解

In [49]:
from hello_agents.tools import MCPTool

# 1. Memory Transport - 内存传输（用于测试）
# 不指定任何参数，使用内置演示服务器
mcp_tool = MCPTool()

# 2. Stdio Transport - 标准输入输出传输（本地开发）
# 使用命令列表启动本地服务器
mcp_tool = MCPTool(server_command=["python", "examples/mcp_example_server.py"])

# 3. Stdio Transport with Args - 带参数的命令传输
# 可以传递额外参数
mcp_tool = MCPTool(server_command=["python", "examples/mcp_example_server.py", "--debug"])

# 4. Stdio Transport - 社区服务器（npx方式）
# 使用npx启动社区MCP服务器
mcp_tool = MCPTool(server_command=["npx", "-y", "@modelcontextprotocol/server-filesystem", "."])

# 5. HTTP/SSE/StreamableHTTP Transport
# 注意：MCPTool主要用于Stdio和Memory传输
# 对于HTTP/SSE等远程传输，建议直接使用MCPClient


INFO:mcp.server.lowlevel.server:Processing request of type ListToolsRequest


🧠 使用内存传输: HelloAgents-BuiltinServer
🔗 连接到 MCP 服务器...
✅ 连接成功！
🔌 连接已断开
📝 使用 Stdio 传输 (命令): python examples/mcp_example_server.py
📝 使用 Stdio 传输 (命令): python examples/mcp_example_server.py --debug
📝 使用 Stdio 传输 (命令): npx -y @modelcontextprotocol/server-filesystem .
🔗 连接到 MCP 服务器...


Exception ignored in: <function StdioTransport.__del__ at 0x7fa70d7d1620>
Traceback (most recent call last):
  File "/mnt/d/anaconda3/envs/flux/lib/python3.12/site-packages/fastmcp/client/transports.py", line 410, in __del__
    if not self._stop_event.is_set():
           ^^^^^^^^^^^^^^^^
AttributeError: 'PythonStdioTransport' object has no attribute '_stop_event'
Exception ignored in: <function StdioTransport.__del__ at 0x7fa70d7d1620>
Traceback (most recent call last):
  File "/mnt/d/anaconda3/envs/flux/lib/python3.12/site-packages/fastmcp/client/transports.py", line 410, in __del__
    if not self._stop_event.is_set():
           ^^^^^^^^^^^^^^^^
AttributeError: 'PythonStdioTransport' object has no attribute '_stop_event'


✅ 连接成功！
🔌 连接已断开


### Memory Transport - 内存传输

In [ ]:
from hello_agents.tools import MCPTool

# 使用内置演示服务器（Memory传输）
mcp_tool = MCPTool()

# 列出可用工具
result = mcp_tool.run({"action": "list_tools"})
print(result)

# 调用工具
result = mcp_tool.run({
    "action": "call_tool",
    "tool_name": "add",
    "arguments": {"a": 10, "b": 20}
})
print(result)


INFO:mcp.server.lowlevel.server:Processing request of type ListToolsRequest
INFO:mcp.server.lowlevel.server:Processing request of type ListToolsRequest
INFO:mcp.server.lowlevel.server:Processing request of type CallToolRequest
INFO:mcp.server.lowlevel.server:Processing request of type ListToolsRequest


🧠 使用内存传输: HelloAgents-BuiltinServer
🔗 连接到 MCP 服务器...
✅ 连接成功！
🔌 连接已断开
🧠 使用内存传输: HelloAgents-BuiltinServer
🔗 连接到 MCP 服务器...
✅ 连接成功！
🔌 连接已断开
找到 6 个工具:
- add: 加法计算器
- subtract: 减法计算器
- multiply: 乘法计算器
- divide: 除法计算器
- greet: 友好问候
- get_system_info: 获取系统信息

🧠 使用内存传输: HelloAgents-BuiltinServer
🔗 连接到 MCP 服务器...
✅ 连接成功！
🔌 连接已断开
工具 'add' 执行结果:
30.0


### Stdio Transport - 标准输入输出传输

In [53]:
from hello_agents.tools import MCPTool

# 方式1：使用自定义Python服务器
# mcp_tool = MCPTool(server_command=["python", "my_mcp_server.py"])

# 方式2：使用社区服务器（文件系统）
mcp_tool = MCPTool(server_command=["npx", "-y", "@modelcontextprotocol/server-filesystem", "."])

# 列出工具
result = mcp_tool.run({"action": "list_tools"})
print(result)

# 调用工具
result = mcp_tool.run({
    "action": "call_tool",
    "tool_name": "read_file",
    "arguments": {"path": "my_README.md"}
})
print(result)


📝 使用 Stdio 传输 (命令): npx -y @modelcontextprotocol/server-filesystem .
🔗 连接到 MCP 服务器...
✅ 连接成功！
🔌 连接已断开
📝 使用 Stdio 传输 (命令): npx -y @modelcontextprotocol/server-filesystem .
🔗 连接到 MCP 服务器...
✅ 连接成功！
🔌 连接已断开
找到 14 个工具:
- read_file: Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.
- read_text_file: Read the complete contents of a file from the file system as text. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Use the 'head' parameter to read only the first N lines of a file, or the 'tail' parameter to read only the last N lines of a file. Operates on the file as text regardless of extension. Only works within allowed directories.
- read_media_file: Read an image or audio file. Returns the base64 encoded data and MIME type. Only works within allowed directories.
- read_multiple_files: Read the contents of multiple files simultaneously. T

### TTP Transport - HTTP 传输

In [54]:
# 注意：MCPTool 主要用于 Stdio 和 Memory 传输
# 对于 HTTP/SSE 等远程传输，建议使用底层的 MCPClient

import asyncio
from hello_agents.protocols import MCPClient

async def test_http_transport():
    # 连接到远程 HTTP MCP 服务器
    client = MCPClient("http://api.example.com/mcp")

    async with client:
        # 获取服务器信息
        tools = await client.list_tools()
        print(f"远程服务器工具: {len(tools)} 个")

        # 调用远程工具
        result = await client.call_tool("process_data", {
            "data": "Hello, World!",
            "operation": "uppercase"
        })
        print(f"远程处理结果: {result}")

# 注意：需要实际的 HTTP MCP 服务器
# asyncio.run(test_http_transport())
